In [1]:
import requests
import json
from pandas import json_normalize
import pandas as pd

In [2]:
from owlready2 import *
onto = get_ontology("http://purl.obolibrary.org/obo/hp.owl").load()
obo = onto.get_namespace("http://purl.obolibrary.org/obo/")
ontology_classes = obo.HP_0000001.descendants()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/BFO_0000023
  http://purl.obolibrary.org/obo/CHEBI_50906

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/CHEBI_36080
  http://purl.obolibrary.org/obo/PR_000000001



In [3]:
o_list = [] 
for current_class in ontology_classes:
    hp_id = current_class.name.replace('_', ':')
    o_list.append(hp_id)

In [4]:
o_100 = o_list[0:100]
s = ",".join(o_100).replace(",", ";")
para = {'q': s}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
df_z = json_normalize(response['results'])
list_z = df_z['concept_id'].values.tolist()
list_z = list(map(str, list_z))
str_z = ",".join(list_z).replace(",", ";")
query = {'dataset_id': 2, 'concept_id': str_z}
info = requests.get('https://rare.cohd.io/api/frequencies/singleConceptFreq', params=query).json()
result = json_normalize(info['results'])

In [5]:
i = 101
while i < 16400: 
    o_100 = o_list[i:i+99]
    s = ",".join(o_100).replace(",", ";")
    para = {'q': s}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
    df_z = json_normalize(response['results'])
    list_z = df_z['concept_id'].values.tolist()
    list_z = list(map(str, list_z))
    str_z = ",".join(list_z).replace(",", ";")
    query = {'dataset_id': 2, 'concept_id': str_z}
    info = requests.get('https://rare.cohd.io/api/frequencies/singleConceptFreq', params=query).json()
    df = json_normalize(info['results'])
    result = pd.concat([result, df])
    i = i+100

o_last = o_list[16401:16480]
s = ",".join(o_last).replace(",", ";")
para = {'q': s}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
df_z = json_normalize(response['results'])
list_z = df_z['concept_id'].values.tolist()
list_z = list(map(str, list_z))
str_z = ",".join(list_z).replace(",", ";")
query = {'dataset_id': 2, 'concept_id': str_z}
info = requests.get('https://rare.cohd.io/api/frequencies/singleConceptFreq', params=query).json()
df = json_normalize(info['results'])
result = pd.concat([result, df])
terms = result.loc[:,("concept_code", "concept_id", "concept_name", "concept_count", "concept_frequency")]
terms

,concept_code,concept_id,concept_name,concept_count,concept_frequency
0,HP:0002039,90002039,Anorexia,15197,0.011258
1,HP:0100651,90100651,Type I diabetes mellitus,12512,0.009269
2,HP:0009800,90009800,Maternal diabetes,12213,0.009048
3,HP:0001622,90001622,Premature birth,11460,0.008490
4,HP:0041055,90041055,Fractured humerus,4573,0.003388
...,...,...,...,...,...
31,HP:0011941,90011941,Anterior wedging of L2,30,0.000022
32,HP:0002340,90002340,Caudate atrophy,16,0.000012
33,HP:0410334,90410334,Fungi allergy,16,0.000012
34,HP:0011742,90011742,Ectopic adrenal gland,11,0.000008


In [ ]:
# (input: dataset, phenotype list, terminology
# output: dic(key:value of risk))

class RiskCalculation:
    def __init__(self, dataset, phenotype_list, terminology):
        self.dataset = dataset
        self.phenotype_list = phenotype_list
        self.terminology = terminology
        
    def calculatePheRS (patient_id, config_file):
        x = getPatientFeatures(patient_id)
        Hyperparameters = getHyperparaFromConfig(congif_file)
        pass
    
    if x in phenotype_list:
        prob = terms['concept_frequency']
    else:
        pass
    log_odds = log(prob/(1-prob))
    
    def Ind_ftn1 (log_odds):
        if log_odds > 0:
            return 1
        else:
            return 0
    
    def Ind_ftn2 (log_odds):
        if log_odds < 0:
            return 1
        else:
            return 0
    
    
    def Dij (x):
        for i in x:
            if i is True:
                return 1
            else:
                return 0
    
    y = (log_odds * Dij * Ind_ftn1) - (log_odds * (1-Dij) * Ind_ftn2)
    risk = sum(y)

    return {'PhRes': risk}
    